# Transacciones

Las transacciones permiten realizar varias operaciones que cambien o consulten la información existente manteniendo la consistencia del sistema.

https://www.sqlitetutorial.net/sqlite-transaction/

Emplearemos un ejemplo sencillo para poder entender porqué existen estos sistemas y es importante distinguirlos de los destinados a analítica.

In [2]:
import sqlite3

# Connect to the DB
conn = sqlite3.connect("transactions.db")
c = conn.cursor()
c.execute("""PRAGMA foreign_keys = ON;""");

Crearemos dos tablas `accounts` y `account_changes` para trabajar con el concepto de las transacciones. No debería suceder que hagamos un traspaso de dinero y falte en una cuenta y no llegue a la otra.

In [33]:
create_stmt = """
DROP TABLE IF EXISTS accounts;
DROP TABLE IF EXISTS account_changes;

CREATE TABLE accounts ( 
	account_no INTEGER NOT NULL, 
	balance DECIMAL NOT NULL DEFAULT 0,
	PRIMARY KEY(account_no),
        CHECK(balance >= 0)
);

CREATE TABLE account_changes (
	change_no INTEGER PRIMARY KEY AUTOINCREMENT,
	account_no INTEGER NOT NULL, 
	flag TEXT NOT NULL, 
	amount DECIMAL NOT NULL, 
	changed_at TEXT NOT NULL 
);

"""
c.executescript(create_stmt);

In [34]:
c.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()

[('sqlite_sequence',), ('accounts',), ('account_changes',)]

Insertaremos datos de ejemplo.

In [35]:
insert_stmt = """
INSERT INTO accounts (account_no,balance)
VALUES (100,20100);

INSERT INTO accounts (account_no,balance)
VALUES (200,10100);

"""
c.executescript(insert_stmt);

Y ahora nos haremos valer de SQLAlchemy para facilitar la consulta a los datos.

In [36]:
# !pip install sqlalchemy

In [37]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///transactions.db', echo=False)

In [38]:
import pandas as pd

acounts_df = pd.read_sql_table("accounts", con=engine)
acounts_df

,account_no,balance
0,100,20100
1,200,10100


¡Todo correcto! Veamos como se realizaría una transacción encargada de mover dinero de una cuenta a otra.

In [39]:
transaction = """

BEGIN TRANSACTION;

UPDATE accounts
   SET balance = balance - 1000
 WHERE account_no = 100;

UPDATE accounts
   SET balance = balance + 1000
 WHERE account_no = 200;
 
INSERT INTO account_changes(account_no,flag,amount,changed_at) 
VALUES(100,'-',1000,datetime('now'));

INSERT INTO account_changes(account_no,flag,amount,changed_at) 
VALUES(200,'+',1000,datetime('now'));

COMMIT;

"""
c.executescript(transaction);

In [40]:
pd.read_sql_table("accounts", con=engine)

,account_no,balance
0,100,19100
1,200,11100


In [41]:
pd.read_sql_table("account_changes", con=engine)

,change_no,account_no,flag,amount,changed_at
0,1,100,-,1000,2024-05-23 16:20:37
1,2,200,+,1000,2024-05-23 16:20:37


Parece que el registro fue bien. ¿Y si intentáramos enviar una cantidad superior a la existente?

In [62]:
amount = 100

try:
    with conn:
        c = conn.cursor()
        c.execute("INSERT INTO account_changes(account_no,flag,amount,changed_at)  VALUES(100,'-',(?),datetime('now'));", (amount,))
        c.execute("UPDATE accounts SET balance = balance - (?) WHERE account_no = 100;", (amount,))
except sqlite3.IntegrityError:
    print("We went to the previous step")

In [63]:
pd.read_sql_table("account_changes", con=engine)

,change_no,account_no,flag,amount,changed_at
0,1,100,-,1000,2024-05-23 16:20:37
1,2,200,+,1000,2024-05-23 16:20:37
2,3,100,-,100,2024-05-23 16:26:30
3,4,100,-,1000,2024-05-23 16:26:56
4,5,100,-,100,2024-05-23 16:27:36


In [64]:
pd.read_sql_table("accounts", con=engine)

,account_no,balance
0,100,17900
1,200,11100


Como veis las transacciones previenen que las operaciones que implican más de un cambio cumplan con los requisitos ACID que buscamos en un sistema que representa nuestro negocio bajo las reglas establecidas.

In [65]:
conn.close()